In [2]:
# iastafev
# LSML 2023

# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


## Upload data to Hadoop

In [1]:
! head /home/jovyan/work/clickstream.csv

user_id	session_id	event_type	event_page	timestamp
562	507	page	main	1695584127
562	507	event	main	1695584134
562	507	event	main	1695584144
562	507	event	main	1695584147
562	507	wNaxLlerrorU	main	1695584154
562	507	event	main	1695584154
562	507	event	main	1695584154
562	507	event	main	1695584160
562	507	page	rabota	1695584166


In [18]:
! hadoop fs -copyFromLocal /home/jovyan/work/clickstream.csv /clickstream.csv

! hadoop fs -ls /

Found 3 items
-rw-r--r--   1 jovyan supergroup   32241574 2024-10-22 14:27 /clickstream.csv
drwxr-xr-x   - root   supergroup          0 2023-09-24 20:38 /data
drwxrwx---   - root   supergroup          0 2024-10-22 14:19 /tmp


## Initialize Spark

In [2]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='sga_task')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-23 11:21:16,437 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


## Load Data

In [3]:
clickstream = se.read.csv("hdfs:///clickstream.csv", header=True, sep='\t',
                      schema='user_id bigint, session_id bigint, event_type string, event_page string, timestamp bigint')
clickstream.registerTempTable("clickstream")

In [156]:
clickstream.show(10)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
+-------+----------+------------+----------+----------+
only showing top 10 rows



In [155]:
clickstream.count()

1000000

## Spark RDD Solution

In [136]:
clickstream.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- session_id: long (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_page: string (nullable = true)
 |-- timestamp: long (nullable = true)



In [157]:
# Extract user path from user session
def session_path(events):
    pages = []
    events.sort(key=lambda x: x.timestamp)
    error_timestamp = None
    prev_page = None
    for i in events:
        if 'error' in i.event_type:
            error_timestamp = i.timestamp
            break # break to save perfomance
        if prev_page is None or i.event_page != prev_page.event_page:
            pages.append(i)
        prev_page = i
    #Filter pages by error ts
    if error_timestamp:
        pages = [page for page in pages if page.timestamp < error_timestamp]
    return '-'.join([page.event_page for page in pages])
        

sessions_rdd = (
    clickstream.rdd
    .map(lambda x: ((x.user_id,x.session_id), x)) #map user+session
    .groupByKey(numPartitions=50) # Group User session Data
    .map(lambda x: (x[0],session_path(x[1].data))) # build user session path
)

paths_rdd = (
    sessions_rdd
    .filter(lambda x: x[1])
    .map(lambda x: (x[1],1))
    .reduceByKey(lambda a, b: a + b)
)

In [163]:
paths_rdd.sortBy(lambda x: x[1], False).toDF(['route','count']).limit(10).show()

+-------------------+-----+
|              route|count|
+-------------------+-----+
|               main| 8184|
|       main-archive| 1113|
|        main-rabota| 1047|
|      main-internet|  897|
|         main-bonus|  870|
|          main-news|  769|
|       main-tariffs|  677|
|        main-online|  587|
|         main-vklad|  518|
|main-rabota-archive|  170|
+-------------------+-----+



In [162]:
rdd_answer = {path: count for (path,count) in paths_rdd.top(10, key=lambda x: x[1])}                 
rdd_answer

{'main': 8184,
 'main-archive': 1113,
 'main-rabota': 1047,
 'main-internet': 897,
 'main-bonus': 870,
 'main-news': 769,
 'main-tariffs': 677,
 'main-online': 587,
 'main-vklad': 518,
 'main-rabota-archive': 170}

## Spark DF solution

In [125]:
from pyspark.sql import Window
import pyspark.sql.functions as F

In [126]:
## Window specs 
window_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

# Add earliest error TS
df_with_error_ts = clickstream.withColumn("error_ts", F.min(F.when(F.col("event_type").contains('error'), F.col("timestamp"))).over(window_spec))

# Add previous page lag
df_with_prev_page = df_with_error_ts.withColumn("prev_page",F.lag("event_page",1).over(window_spec))

# Filter only page changes
df_page_changed = df_with_prev_page.where((F.col('event_page') != F.col('prev_page')) | (F.col('prev_page').isNull()))

# Filter by session errors
df_filtered = df_page_changed.where((F.col('error_ts') > F.col('timestamp')) | (F.col('error_ts').isNull()))

# Build session route
df_session_routes = df_filtered.groupBy("user_id","session_id").agg(
    F.concat_ws("-", F.collect_list("event_page")).alias("route")
)

# Session Counts
df_answer = df_session_routes.groupBy('route').count().sort(F.desc('count')).limit(10)

df_answer.show()

+-------------------+-----+
|              route|count|
+-------------------+-----+
|               main| 8184|
|       main-archive| 1112|
|        main-rabota| 1047|
|      main-internet|  897|
|         main-bonus|  870|
|          main-news|  769|
|       main-tariffs|  676|
|        main-online|  587|
|         main-vklad|  518|
|main-rabota-archive|  170|
+-------------------+-----+



## Spark SQl solution

In [154]:
sql_query = """
WITH with_error_ts AS (
  SELECT
    *,
    MIN(CASE WHEN event_type LIKE '%error%' THEN timestamp END) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS error_ts
  FROM clickstream
),
clickstream_with_prev_page AS (
  SELECT 
    *,
    LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as prev_page
  FROM with_error_ts
),
clickstream_filtered AS (
  SELECT 
    *
  FROM clickstream_with_prev_page
  WHERE (timestamp < error_ts OR error_ts IS NULL) AND (prev_page IS NULL OR event_page != prev_page)
),
session_routes AS (
  SELECT 
    user_id,
    session_id,
    concat_ws('-', collect_list(event_page)) AS route
  FROM clickstream_filtered
  GROUP BY user_id,session_id
),
session_counts AS (
  SELECT
    route,
    COUNT(*) AS count
  FROM session_routes
  GROUP BY route
)
SELECT * FROM session_counts ORDER BY count DESC LIMIT 10
"""
sql_answer = se.sql(sql_query)
sql_answer.show()

+-------------------+-----+
|              route|count|
+-------------------+-----+
|               main| 8184|
|       main-archive| 1112|
|        main-rabota| 1047|
|      main-internet|  897|
|         main-bonus|  870|
|          main-news|  769|
|       main-tariffs|  676|
|        main-online|  587|
|         main-vklad|  518|
|main-rabota-archive|  170|
+-------------------+-----+

